In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split
# building a alexnet model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping




In [2]:
# Load the CSV file
data = pd.read_csv('merged_less_up.csv')


# print unique values for action
print(data['action'].unique())

# replace nan values with 'no_action'
# data['action'] = data['action'].fillna('no_action')

# drop nan values
data.dropna(inplace=True)


# print unique values for action
# print(data['action'].unique())

print(data['action'].value_counts())
print(data.tail(10))

['up' 'right' nan 'left' 'down']
action
up       4851
right    4613
left     4041
down     1701
Name: count, dtype: int64
                          image_name last_action action
20349  11301_2024-06-01_17-34-01.png        left   left
20350  11302_2024-06-01_17-34-01.png        left   left
20351  11303_2024-06-01_17-34-01.png        left   left
20355  11313_2024-06-01_17-34-01.png        left   left
20356  11314_2024-06-01_17-34-01.png        left   left
20357  11315_2024-06-01_17-34-01.png        left   left
20359  11329_2024-06-01_17-34-01.png          up     up
20360  11330_2024-06-01_17-34-01.png          up     up
20361  11333_2024-06-01_17-34-01.png          up     up
20362  11347_2024-06-01_17-34-01.png          up     up


In [3]:
mapping = {'up': 0, 'left': 1, 'right': 2, 'down': 3}

def encode_action(action):
    ac = mapping[action]
    temp=np.zeros(4)
    temp[ac]=1
    return temp

data["encoded_action"] = data["action"].apply(encode_action)


In [4]:
data.tail(10)

,image_name,last_action,action,encoded_action
20349,11301_2024-06-01_17-34-01.png,left,left,"[0.0, 1.0, 0.0, 0.0]"
20350,11302_2024-06-01_17-34-01.png,left,left,"[0.0, 1.0, 0.0, 0.0]"
20351,11303_2024-06-01_17-34-01.png,left,left,"[0.0, 1.0, 0.0, 0.0]"
20355,11313_2024-06-01_17-34-01.png,left,left,"[0.0, 1.0, 0.0, 0.0]"
20356,11314_2024-06-01_17-34-01.png,left,left,"[0.0, 1.0, 0.0, 0.0]"
20357,11315_2024-06-01_17-34-01.png,left,left,"[0.0, 1.0, 0.0, 0.0]"
20359,11329_2024-06-01_17-34-01.png,up,up,"[1.0, 0.0, 0.0, 0.0]"
20360,11330_2024-06-01_17-34-01.png,up,up,"[1.0, 0.0, 0.0, 0.0]"
20361,11333_2024-06-01_17-34-01.png,up,up,"[1.0, 0.0, 0.0, 0.0]"
20362,11347_2024-06-01_17-34-01.png,up,up,"[1.0, 0.0, 0.0, 0.0]"


In [5]:

# Load images and corresponding labels
images = []
labels = []

for idx, row in data.iterrows():
    img_path = os.path.join('merged', row['image_name'])
    img = load_img(img_path,color_mode="grayscale")  # Resize images to 256x144
    img_array = img_to_array(img) / 255.0
    images.append(img_array)
    labels.append(row['encoded_action'])

# Convert lists to numpy arrays
X = np.array(images)
y = np.array(labels)


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the alextnet model architecture

In [15]:



def alexNet():
    inp = tf.keras.layers.Input(shape=(144, 256, 1))
    # 1st Convolutional Layer
    x = Conv2D(filters=96, input_shape=(144, 256, 1), kernel_size=(11, 11), strides=(4, 4), padding='same',
               activation='relu')(inp)
    # Max Pooling
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    # 2nd Convolutional Layer
    x = Conv2D(filters=256, kernel_size=(11, 11), strides=(1, 1), padding='same', activation='relu')(x)
    # Max Pooling
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    # 3rd Convolutional Layer
    x = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)

    # 4th Convolutional Layer
    x = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)

    # 5th Convolutional Layer
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    # Max Pooling
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)

    # Passing it to a Fully Connected layer
    x = Flatten()(x)
    # 1st Fully Connected Layer
    x = Dense(units=4096, activation='relu')(x)
    # Add Dropout to prevent overfitting
    x = Dropout(0.4)(x)

    # 2nd Fully Connected Layer
    x = Dense(units=4096, activation='relu')(x)
    # Add Dropout
    x = Dropout(0.4)(x)

    # 3rd Fully Connected Layer
    x = Dense(units=1000, activation='relu')(x)
    # Add Dropout
    x = Dropout(0.4)(x)

    # Output Layer
    x = Dense(units=4, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=x)
    return model




In [16]:

model = alexNet()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 144, 256, 1)]     0         
                                                                 
 conv2d_13 (Conv2D)          (None, 36, 64, 96)        11712     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 18, 32, 96)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 18, 32, 256)       2973952   
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 9, 16, 256)       0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 9, 16, 384)        8851

In [17]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)



In [18]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/100
381/381 [==============================] - 22s 42ms/step - loss: 1.3278 - accuracy: 0.3081 - val_loss: 1.3045 - val_accuracy: 0.3353
Epoch 2/100
381/381 [==============================] - 15s 40ms/step - loss: 1.2563 - accuracy: 0.3859 - val_loss: 1.2140 - val_accuracy: 0.4467
Epoch 3/100
381/381 [==============================] - 15s 40ms/step - loss: 1.1009 - accuracy: 0.5187 - val_loss: 0.9587 - val_accuracy: 0.5976
Epoch 4/100
381/381 [==============================] - 15s 39ms/step - loss: 0.7961 - accuracy: 0.6829 - val_loss: 0.7121 - val_accuracy: 0.7206
Epoch 5/100
381/381 [==============================] - 15s 39ms/step - loss: 0.5704 - accuracy: 0.7885 - val_loss: 0.5772 - val_accuracy: 0.7840
Epoch 6/100
381/381 [==============================] - 15s 39ms/step - loss: 0.4462 - accuracy: 0.8394 - val_loss: 0.5618 - val_accuracy: 0.8031
Epoch 7/100
381/381 [==============================] - 15s 39ms/step - loss: 0.3385 - accuracy: 0.8848 - val_loss: 0.5335 - val_ac

In [19]:
# evaluate the model
loss, acc = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {acc}")
print(f"Validation loss: {loss}")


96/96 [==============================] - 1s 13ms/step - loss: 0.5149 - accuracy: 0.8337
Validation accuracy: 0.8336620926856995
Validation loss: 0.5149314999580383


In [20]:

# Save the model
model.save('alexnet.h5')